# Theoretical EoS Prior Visualization

## Detailed Analysis

This script is used to visualize the full parameter space of the Equations of State (EoS) generated for the project. Unlike other plotting scripts that visualize the resulting stars (Mass-Radius), this script focuses on the underlying thermodynamics: the relationship between Pressure ($P$) and Energy Density ($\epsilon$).

By generating and plotting a representative sample of curves (defined by `n_curves`), the script illustrates the "theoretical prior" or the range of physical models covered by the simulation. It highlights the distinct behaviors of Hadronic matter (Green) and Quark matter (Magenta) and compares them against fundamental physical limits.

## Physics and Math

The plot visualizes the EoS function $P(\epsilon)$. Two theoretical limits are drawn for reference, representing the stiffness of the matter (the speed of sound squared $c_s^2 = dP/d\epsilon$):

### 1. Causal Limit
The speed of sound in matter cannot exceed the speed of light ($c_s \le 1$). The stiffest possible EoS is therefore:
$$
P = \epsilon
$$
This appears as a line with a slope of 1 on the log-log plot.

### 2. Conformal Limit
In the limit of extremely high densities, Quantum Chromodynamics (QCD) predicts that matter becomes conformally symmetric, and the trace of the energy-momentum tensor vanishes. This leads to:
$$
P = \frac{1}{3}\epsilon \quad \Rightarrow \quad c_s^2 = \frac{1}{3}
$$
This is often treated as a "soft" upper bound for the stiffness of core matter in some theories.

## Code Walkthrough

### 1. Parallel Curve Generation
To ensure the visualization accurately reflects the training data, the script uses the same generation logic as the main pipeline. It spawns parallel workers to generate `n_curves` for both populations.

```python
# Run Workers
res_h = Parallel(n_jobs=-1)(delayed(worker_get_plot_curve)('hadronic', baselines, s) for s in seeds)
res_q = Parallel(n_jobs=-1)(delayed(worker_get_plot_curve)('quark', baselines, s) for s in seeds)
```

The function `worker_get_plot_curve` is used here because it returns the dense microscopic $P-\epsilon$ grid, whereas the standard workers return only macroscopic features.

### 2. Plotting the Bands
The generated curves are plotted on a log-log scale. A low transparency value (`alpha`) is used.
*   **Hadronic Models:** Plotted in Green. They typically extend to lower energy densities (representing the crust).
*   **Quark Models:** Plotted in Magenta. They terminate abruptly at low densities due to the vacuum pressure $B$ (where $P=0$ but $\epsilon \approx 4B$).

### 3. Constraints and Formatting
The physical limits are added to the plot to frame the model space.

```python
# Causal Limit (P = epsilon)
ax.plot(x_guide, x_guide, color='black', linestyle=':', ...)

# Conformal Limit (P = epsilon/3)
ax.plot(x_guide, x_guide/3.0, color='gray', linestyle='--', ...)
```

The axes are set to logarithmic scales, and the limits are strictly enforced using `CONSTANTS['PLOT_EPS_LOG']` to ensure the plot focuses on the core density region relevant to neutron stars.

## Visualization Output

The script saves a single PDF file: `plots/fig_theoretical_eos.pdf`.

*   **X-Axis:** Energy Density $\epsilon$ [MeV/fm$^3$] (Log scale).
*   **Y-Axis:** Pressure $P$ [MeV/fm$^3$] (Log scale).
*   **Visual Features:**
    *   **Hadronic Band (Green):** Shows a continuous evolution from low density.
    *   **Quark Band (Magenta):** Starts at finite density ($\sim 200-500$ MeV/fm$^3$).
    *   **Stiffness:** The slope of the curves represents the stiffness. Quark models often run parallel to the Causal limit (very stiff) before bending, while Hadronic models show more curvature.